In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.2
!pip uninstall keras
!pip install keras==2.1.2
!pip uninstall numpy
!pip install numpy==1.19

Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.2+zzzcolab20220719082949.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 516.2 MB 4.5 kB/s 
     |████████████████████████████████| 3.0 MB 47.6 MB/s 
     |████████████████████████████████| 26.1 MB 1.2 MB/s 
     |████████████████████████████████| 454 kB 66.4 MB/s 
     |███████

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
#import packages

import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
import tensorflow as tf
from sklearn.model_selection import train_test_split
import unicodedata
import io
import time



Using TensorFlow backend.


In [ ]:
path_to_file = "/content/drive/MyDrive/Dataset/Train_new.txt"
# Preview the dataset
lines = io.open(path_to_file, encoding='utf-8').read().strip().split('\n')

[[w for w in l.split('\t')[:2]]  for l in lines[:5]]

[['عليك مراجعه جراح عام وعمل صوره رنين مغناطيسي للبطن والتاكد من وضع القنوات الصفراويه احيانا يحصل الالم بعد عمليه المنظار بسبب وجود الهواء في البطن ويزول مع الوقت',
  'اللي بيحس بالم بعد عمليه استئصال مراره سببه ايه بيكون الم في منطقه المراره وفي اسفل الجهه اليمنى من البطن'],
 ['لا علاقه للسكر او الكلسترول بهذه الالام اما باقي فقد تمت الاجابه عنه في نفس المكرر',
  'قريبتي تشتكي من الام اسفل الظهر بقرب عظمه الحوض وتشعر ايضا بنغزات في الثدي مااسباب هذا الالم وهل للسكر والكلسترول سبب في هذا'],
 ['النهايات العصبيه في الاطراف العلويه والسفليه دقيقه وعند البعض تتاثر بالبروده وعند اخرين بالسخونه وخاصه لدى مرضى السكري او من يعانون من اعتلال عصبي محيطي اذا كانت الحاله مزعجه بشكل كبير يمكن عمل تخطيط اعصاب للاطراف العلويه للتاكد من عدم وجود انضغاط عصبي في اليدين',
  'اشعر بتنميل في اطراف الاصابع عندما تلامس المياه وخصوصا المياه الساخنه فما السبب'],
 ['قد يكون ما تعاني منه هو حب الشباب الدهني ولكن لابد من الفحص السرير لتاكيد التشخيص',
  'اريد الاستفسار عن الحبوب على الجسم على الظهر و الصدر و اليد

In [ ]:
word_pairs = [[w for w in l.split('\t')[:2]]  for l in lines]

In [ ]:
len(word_pairs)

10233

In [ ]:
texts_1 = [word_pairs[i][0] for i in range(len(word_pairs))]
texts_2 = [word_pairs[i][-1] for i in range(len(word_pairs))]

In [ ]:
texts_1[1]

'لا علاقه للسكر او الكلسترول بهذه الالام اما باقي فقد تمت الاجابه عنه في نفس المكرر'

In [ ]:
texts_2[1]

'قريبتي تشتكي من الام اسفل الظهر بقرب عظمه الحوض وتشعر ايضا بنغزات في الثدي مااسباب هذا الالم وهل للسكر والكلسترول سبب في هذا'

In [ ]:
#PreTreatment

def preTreatement(w):
  
  w = w.lower().strip()

  # unicode to ascii
  w = ''.join(c for c in unicodedata.normalize('NFD', w) if unicodedata.category(c) != 'Mn')
  #delete extra white space at the end & begining
  w = w.strip()

  # add the start and end tokens
  w = '<start> ' + w + ' <end>'
    
  return w

In [ ]:
# making the pre treatement to the sentences, and return the dataset in the format : answers, questions
def extractSentences():

  answer = [preTreatement(w) for w in texts_1]
  question = [preTreatement(w) for w in texts_2]


  return answer, question

In [ ]:
#Tokenization

def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')

  return tensor, lang_tokenizer

In [ ]:
# Load the dataset + Tokenization for the inputs and outputs
#input= Answer
#target= Question
def loadSentences():
    
  # creating cleaned input, output pairs
  inputLang, targetLang = extractSentences()

  inputTensor, inputLang_tokenizer = tokenize(inputLang)
  targetTensor, targetLang_tokenizer = tokenize(targetLang)

  return inputTensor, targetTensor, inputLang_tokenizer, targetLang_tokenizer

In [ ]:
# Load input and output tensors

inputTensor, targetTensor, inputLang, targetLang = loadSentences()

# Calculate max_length of the target tensors
maxLengthTarget, maxLengthInput = targetTensor.shape[1], inputTensor.shape[1]

In [ ]:
# Creating training and validation sets using an 80-20 split (you must also leave another 20% aside for the test ... we have left this 20% in a separate file)

#inputTensor_train, inputTensor_val, targetTensor_train, targetTensor_val = train_test_split(inputTensor, targetTensor, test_size=0.2)
inputTensor_train = inputTensor
targetTensor_train = targetTensor
# Show length
print(len(inputTensor_train), len(targetTensor_train))

10233 10233


In [ ]:
#Convert from index to word

def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [ ]:
#show an example
#input= Answer
#target= Question
print ("Input (ANSWER); index to word mapping")
convert(inputLang, inputTensor_train[0])
print ()
print ("Target (QUESTION); index to word mapping")
convert(targetLang, targetTensor_train[0])

Input (ANSWER); index to word mapping
1 ----> <start>
61 ----> عليك
23 ----> مراجعه
129 ----> جراح
214 ----> عام
196 ----> وعمل
114 ----> صوره
440 ----> رنين
667 ----> مغناطيسي
1137 ----> للبطن
880 ----> والتاكد
3 ----> من
252 ----> وضع
2437 ----> القنوات
3837 ----> الصفراويه
362 ----> احيانا
621 ----> يحصل
79 ----> الالم
21 ----> بعد
107 ----> عمليه
881 ----> المنظار
87 ----> بسبب
33 ----> وجود
2281 ----> الهواء
4 ----> في
215 ----> البطن
4273 ----> ويزول
17 ----> مع
283 ----> الوقت
2 ----> <end>

Target (QUESTION); index to word mapping
1 ----> <start>
968 ----> اللي
12576 ----> بيحس
118 ----> بالم
13 ----> بعد
41 ----> عمليه
351 ----> استيصال
3807 ----> مراره
1091 ----> سببه
563 ----> ايه
2515 ----> بيكون
21 ----> الم
4 ----> في
175 ----> منطقه
508 ----> المراره
189 ----> وفي
95 ----> اسفل
245 ----> الجهه
134 ----> اليمنى
3 ----> من
105 ----> البطن
2 ----> <end>


In [ ]:
#Definition of hyper Parameters

bufferSize = len(inputTensor_train)
batchSize = 64
stepsPerEpoch = len(inputTensor_train)//batchSize
embeddingDimension = 256
units = 1024
vocabInputSize = len(inputLang.word_index)+1
vocabTargetSize = len(targetLang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((inputTensor_train, targetTensor_train)).shuffle(bufferSize)
dataset = dataset.batch(batchSize, drop_remainder=True)

#example of input and target batches

exampleInputBatch, exampleTargetBatch = next(iter(dataset))
exampleInputBatch.shape, exampleTargetBatch.shape

(TensorShape([64, 52]), TensorShape([64, 52]))

In [ ]:
# Encoder class

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embeddingDimension, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embeddingDimension)
    self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocabInputSize, embeddingDimension, units, batchSize)

sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(exampleInputBatch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 52, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
#Attention mechanism class

class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    
    query_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

    attention_weights = tf.nn.softmax(score, axis=1)

    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batchSize, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batchSize, sequence_length, 1) (64, 52, 1)


In [ ]:
#Decoder class

class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embeddingDimension, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embeddingDimension)
    self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    

    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    
    context_vector, attention_weights = self.attention(hidden, enc_output)

    x = self.embedding(x)
    
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    output, state = self.gru(x)

    output = tf.reshape(output, (-1, output.shape[2]))
    
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocabTargetSize, embeddingDimension, units, batchSize)

sample_decoder_output, _, _ = decoder(tf.random.uniform((batchSize, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batchSize, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batchSize, vocab size) (64, 27057)


In [ ]:
print(vocabTargetSize)

27057


In [ ]:
print(vocabInputSize)

24976


In [ ]:
#Define the optimizer and the loss function

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
#Checkpoints 

checkpoint_dir = '/content/drive/MyDrive/train_file/28'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=2)

In [ ]:
#Training function

@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
        
    # return the encoder output and the decoder hidden state
    
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targetLang.word_index['<start>']] * batchSize, 1)

    # giving the target as the next input (teacher forcing)
    
    for t in range(1, targ.shape[1]):
        
      # passing encoder output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      dec_input = tf.expand_dims(targ[:, t], 1)

  #calculate the loss

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  #calculate the gradients (for backpropagation + optimizer)

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
checkpoint.restore(checkpoint_manager .latest_checkpoint)
if checkpoint_manager .latest_checkpoint:
  print("Restored from {}".format(checkpoint_manager.latest_checkpoint))

else:
  print("Initializing from scratch.")

Restored from /content/drive/MyDrive/train_file/27/ckpt-30


In [ ]:
# Run the training 

# restore the latest checkpoint 
#checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

EPOCHS = 30 
start_epoch = 0
tic = time.clock()
if checkpoint_manager.latest_checkpoint:
  start_epoch = int(checkpoint_manager.latest_checkpoint.split('-')[-1])
  checkpoint.restore(checkpoint_manager.latest_checkpoint)


for epoch in range(start_epoch,EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(stepsPerEpoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
    
  # saving a checkpoint of the model after every epoch
  
  checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / stepsPerEpoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

  #Save loss value in a txt file after every epoch (Optional)
  
  loss_file=open("/content/drive/MyDrive/Final_loss_NEW.txt", "a+")
  loss_file.write(str(total_loss / stepsPerEpoch)+'\n')
  loss_file.close()
  
  print("Model and loss saved\n")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


Epoch 27 Batch 0 Loss 0.1641
Epoch 27 Batch 100 Loss 0.1901
Epoch 27 Loss 0.1867
Time taken for 1 epoch 135.40615272521973 sec

Model and loss saved

Epoch 28 Batch 0 Loss 0.1313
Epoch 28 Batch 100 Loss 0.1984
Epoch 28 Loss 0.1530
Time taken for 1 epoch 135.6809196472168 sec

Model and loss saved

Epoch 29 Batch 0 Loss 0.1384
Epoch 29 Batch 100 Loss 0.1127
Epoch 29 Loss 0.1253
Time taken for 1 epoch 135.2557783126831 sec

Model and loss saved

Epoch 30 Batch 0 Loss 0.0756
Epoch 30 Batch 100 Loss 0.1007
Epoch 30 Loss 0.1029
Time taken for 1 epoch 135.61100697517395 sec

Model and loss saved



In [ ]:
# Passing to the inference

# restore the latest checkpoint 
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
#Generate Questions

def evaluate(sentence):

  sentence = preTreatement(sentence)

  inputs = [inputLang.word_index[i] for i in sentence.split(' ') if i in inputLang.word_index.keys()]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=maxLengthInput, padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targetLang.word_index['<start>']], 0)

  for t in range(maxLengthTarget):
        
    predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targetLang.index_word[predicted_id] + ' '

    if targetLang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)
    
  return result, sentence

In [ ]:
def generate_question(sentence):
    
  result, sentence = evaluate(sentence)
  f_res = result[:-6].rstrip()
  return f_res